<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 6:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [184]:
# PREPARE REQUIRED LIBRARIES
import pandas as pd
import numpy as np
#import pandas.Serie as pd_series


from scrapy.selector import Selector
from scrapy.http import HtmlResponse

import requests

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [28]:
# SCRAPE WIKIPEDIA FOR LARGEST US CITIES (NON-EXHAUSTIVE LIST)
# Found this site instead w the top ten cities. 
response = requests.get("http://www.infoplease.com/toptens/largestcities.html")
HTML = response.text  
HTML[0:150]

u'<!DOCTYPE html>\n<html>\n<head>\n\n<link rel="icon" href="http://i.infopls.com/favicon.ico" type="image/x-icon"/>\n<link rel="shortcut icon" href="http://i'

In [34]:
# //tr/td[2]/a for wikipedia

cities = Selector(text=HTML).xpath("//td[@class='ttcontent'][2]/text()").extract()
df = pd.DataFrame(cities)
df

,0
0,"New York, N.Y."
1,"Los Angeles, Calif."
2,"Chicago, Ill."
3,"Houston, Tex."
4,"Philadelphia, Pa."
5,"Phoenix, Ariz."
6,"San Antonio, Tex."
7,"San Diego, Calif."
8,"Dallas, Tex."
9,"San Jose, Calif."


In [44]:
df[0]

0         New York, N.Y.
1    Los Angeles, Calif.
2          Chicago, Ill.
3          Houston, Tex.
4      Philadelphia, Pa.
5         Phoenix, Ariz.
6      San Antonio, Tex.
7      San Diego, Calif.
8           Dallas, Tex.
9       San Jose, Calif.
Name: 0, dtype: object

In [107]:
def city_only(df):
    df_city = []
    for x in range(0,len(df[0])):
        (x_city, x_State) = df[0][x].split(',') 
        #print x_city, df[0][x]
        df_city.append(x_city)

    return df_city
 
cities = city_only(df)
print type(cities)

<type 'list'>


In [113]:
# use replace to get rid of spaces, change to lower case of cities
cities_simpler_lower = [x.lower().replace(' ','') for x in cities]
cities_simpler_lower

[u'newyork',
 u'losangeles',
 u'chicago',
 u'houston',
 u'philadelphia',
 u'phoenix',
 u'sanantonio',
 u'sandiego',
 u'dallas',
 u'sanjose']

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [2]:
# ONLY RETAIN PROPERLY FORMED CITIES WITH FILTERING FUNCTION


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [136]:
def rv_info(city):
    
    rv_url1 = "http://"+city+".craigslist.org/search/sss?query=rv"
    print rv_url1
    response1 = requests.get(rv_url1)
    HTML1 = response1.text  
    HTML1[0:150]

    rv_prices1 = Selector(text=HTML1).xpath("//span[@class='price']/text()").extract()
    df_prices1 = pd.DataFrame(rv_prices1)

    return df_prices1

In [140]:
df = rv_info("sanfrancisco")
df.head()

http://sanfrancisco.craigslist.org/search/sss?query=rv


,0
0,$44995
1,$44995
2,$17999
3,$17999
4,$94995


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [212]:
city_map  = {
    "sanfrancisco":"sfbayarea",
    "sfbayarea":"sfbayarea",
    "newyork": "newyork",
    "nyc": "newyork",
    "losangeles": "losangeles",
    "la":"losangeles",
    "chicago":"chicago",
    "metro_chicago":"chicago",
    "houston":"houston",
    "philadelphia":"philadelphia",
    "philly":"philadelphia",
    "phoenix":"phoenix",
    "sanantonio":"sanantonio",
    "sandiego":"sandiego",
    "dallas":"dallas",
    "the_big_d":"dallas",
    "sanjose":"sfbayarea"
}

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [210]:
def mean_med(rvs):
    
    df2_clean = [float(x.replace('$','')) for x in rvs[0]]
    
    print "mean is:", sum(df2_clean) /float(len(df2_clean))
    
    print "median is:", np.median(np.array(df2_clean))
    

In [213]:

for city  in city_map:
    
    df3 = rv_info(city_map[city])
    print city
    mean_med(df3)



http://houston.craigslist.org/search/sss?query=rv
houston
mean is: 38804.7142857
median is: 26053.0
http://chicago.craigslist.org/search/sss?query=rv
metro_chicago
mean is: 11078.1111111
median is: 700.0
http://losangeles.craigslist.org/search/sss?query=rv
losangeles
mean is: 62654.344086
median is: 15995.0
http://sfbayarea.craigslist.org/search/sss?query=rv
sanfrancisco
mean is: 25293.5540541
median is: 20499.0
http://sandiego.craigslist.org/search/sss?query=rv
sandiego
mean is: 15934.164557
median is: 280.0
http://phoenix.craigslist.org/search/sss?query=rv
phoenix
mean is: 30047.923913
median is: 26465.0
http://sfbayarea.craigslist.org/search/sss?query=rv
sfbayarea
mean is: 25293.5540541
median is: 20499.0
http://newyork.craigslist.org/search/sss?query=rv
newyork
mean is: 13687.0595238
median is: 1600.0
http://losangeles.craigslist.org/search/sss?query=rv
la
mean is: 62654.344086
median is: 15995.0
http://philadelphia.craigslist.org/search/sss?query=rv
philadelphia
mean is: 20019.112

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Run your scraping process, and save your results to a CSV file.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._